### Tasks: 

    1) Read the MBA file for 2015, pre-process it to generate word embedding

    2) Baseline: Machine translation (autoencoder) method to predict products for a transaction based on most recent transaction by a consumer. Add user features into model as well.
    
    3) Evaluate predictive performance 

In [41]:
import pandas as pd
import numpy as np
import os
import gensim

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [18]:


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
from keras import regularizers

/Users/karthiksrinivasan/DD/Python/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# print(os.getcwd())
trans = pd.read_csv("../data/purchases_panel_prod_trips_2015_new.csv", nrows=1000000)
#trans = pd.read_csv("../data/purchases_panel_prod_trips_2015_new.csv")
print(trans.shape)

(1000000, 51)


In [3]:
#trans_temp = pd.read_csv("../data/purchases_panel_prod_trips_2015_new.csv", nrows=1000)
#trans_temp.to_csv('Check_data_sample.csv')

In [4]:
print(trans.head())
print(trans.columns)

   Age And Presence Of Children  DMA Cd       F1  Female Head Age  \
0                             9     510  3789730                9   
1                             3     561   727313                5   
2                             3     602  5255228                7   
3                             9     504   170207                8   
4                             9     641  3226111                8   

   Fips County Cd  Fips State Cd  Household Code  Household Composition  \
0             169             39         9068070                      1   
1             125             12         6993068                      2   
2              89             18        30343387                      1   
3              11             42         2059554                      1   
4              29             48         8671061                      1   

   Household Income  Household Size     ...      Store Zip3  Total Price Paid  \
0                16               2     ...          

In [5]:
trans['rowno'] = range(trans.shape[0])
trans['tripcode'] = trans['Trip Code Uc']

In [6]:
print(trans[['Scantrack Market Identifier Cd']].nunique())
print(trans[['DMA Cd']].nunique())
print(trans[['F1']].nunique())
print(trans[['Household Code']].nunique())
print(trans[['Trip Code Uc']].nunique())
print(trans.tripcode.nunique())
print(trans['rowno'].nunique())
# print(trans[['Trip Code Uc','Upc']].nunique())

Scantrack Market Identifier Cd    76
dtype: int64
DMA Cd    203
dtype: int64
F1    517006
dtype: int64
Household Code    8484
dtype: int64
Trip Code Uc    517006
dtype: int64
517006
1000000


In [6]:
trans["Upc"] = trans["Upc"].astype(str)
trans_products = trans.groupby("tripcode").apply(lambda trans: trans['Upc'].tolist())


In [8]:
print(trans_products.shape)
print(trans_products.head())

(517006,)
tripcode
1055365928                             [30024053785]
1055365934    [89445500031, 7724150070, 70708210032]
1055365992                              [4173601028]
1055365998                              [3400000480]
1055366033      [3400008752, 3400008752, 4000015122]
dtype: object


In [9]:
trans_products2 = trans.groupby(["Purchase Date", "tripcode"]).apply(lambda trans: trans['Upc'].tolist())
print(trans_products2.shape)
print(trans_products2.head())

(517006,)
Purchase Date  tripcode  
1/1/2015       1055382666                                         [9955515593]
               1055382862    [4133102702, 4133102702, 4133102702, 714290984...
               1055388216                            [81106801000, 4127102505]
               1055390694                                         [1300000112]
               1055446013                                         [3620001401]
dtype: object


In [11]:
longest = np.max(trans_products.apply(len))
print(longest)

42


In [13]:
trans_products = trans_products.values
word2vec_mb = gensim.models.Word2Vec(trans_products, sg = 1, size=300, window=longest, min_count=10, workers=4)

In [14]:
# We group by date and Household number as an alternative

# house_products = trans.groupby(["Purchase Date", "Household Code"], as_index=False).apply(lambda trans: trans['Upc'].tolist())
house_products = pd.DataFrame(trans.groupby(["Purchase Date", "Household Code"]).apply(lambda trans: trans['Upc'].tolist()).reset_index(name = "List_UPCs"))
print(house_products.shape)
print(house_products.head())

(459689, 3)
  Purchase Date  Household Code      List_UPCs
0      1/1/2015         2002813   [5210000043]
1      1/1/2015         2016928   [7754429780]
2      1/1/2015         2021427  [78535720610]
3      1/1/2015         2024288   [3400012116]
4      1/1/2015         2029790   [7673712236]


In [15]:
house_prd = house_products.copy()
house_prd.columns = ["PurchaseDate", "HouseholdCode", "UPCs"]
house_prd = house_prd[["HouseholdCode", "PurchaseDate", "UPCs"]]
house_prd = house_prd.sort_values(by=["HouseholdCode", "PurchaseDate"])
print(house_prd.head())

       HouseholdCode PurchaseDate  \
20274        2001500    1/24/2015   
24570        2001500    1/27/2015   
32831        2001500     1/4/2015   
47976        2001500   10/15/2015   
53326        2001500   10/19/2015   

                                                    UPCs  
20274   [3890000513, 7222300011, 7248600220, 3620000550]  
24570                                       [2220000491]  
32831               [2200000665, 2200000665, 3500076221]  
47976  [7017726772, 3291700013, 4100000287, 340000426...  
53326                                       [3890000409]  


In [8]:
### We now aggregate the list into a weekly format per customer
## Code reference: https://stackoverflow.com/questions/45281297/group-by-week-in-pandas

trans['Date'] = pd.to_datetime(trans["Purchase Date"]) - pd.to_timedelta(7, unit='d')
house_prd_weekly = pd.DataFrame(trans.groupby(["Household Code", pd.Grouper(key='Date', freq='W-MON')]).apply(lambda trans: trans['Upc'].tolist()).reset_index(name = "List_UPCs"))

house_prd_weekly.columns = [ "HouseholdCode", "PurchaseWeek", "List_UPCs"]
house_prd_weekly = house_prd_weekly.sort_values(by=["HouseholdCode", "PurchaseWeek"])

print (house_prd_weekly.shape)
print (house_prd_weekly.head())

(309001, 3)
   HouseholdCode PurchaseWeek  \
0        2001500   2014-12-22   
1        2001500   2014-12-29   
2        2001500   2015-01-19   
3        2001500   2015-01-26   
4        2001500   2015-02-02   

                                           List_UPCs  
0                                       [4300028591]  
1               [2200000665, 2200000665, 3500076221]  
2   [3890000513, 7222300011, 7248600220, 3620000550]  
3                                       [2220000491]  
4  [2200000668, 2200000665, 2200000665, 300653144...  


In [39]:
#temp = [house_prd_weekly.List_UPCs[1:(len(house_prd_weekly.List_UPCs))],house_prd_weekly.List_UPCs[(len(house_prd_weekly.List_UPCs))]
#house_prd_weekly.FutureUPCs = temp
house_prd_weekly["FutureUPCs"] = house_prd_weekly.List_UPCs.shift(-1)
house_prd_weekly["FutureUPCs"].iloc[-1] = house_prd_weekly["List_UPCs"].iloc[-1] 

#print(house_prd_weekly.FutureUPCs.head())
#print(house_prd_weekly.FutureUPCs.tail())
#print("Past")
#print(house_prd_weekly.List_UPCs.head())
#print(house_prd_weekly.List_UPCs.tail())

print(house_prd_weekly.head())
print(house_prd_weekly.tail())
print(house_prd_weekly.shape)

   HouseholdCode PurchaseWeek  \
0        2001500   2014-12-22   
1        2001500   2014-12-29   
2        2001500   2015-01-19   
3        2001500   2015-01-26   
4        2001500   2015-02-02   

                                           List_UPCs  \
0                                       [4300028591]   
1               [2200000665, 2200000665, 3500076221]   
2   [3890000513, 7222300011, 7248600220, 3620000550]   
3                                       [2220000491]   
4  [2200000668, 2200000665, 2200000665, 300653144...   

                                          FutureUPCs  
0               [2200000665, 2200000665, 3500076221]  
1   [3890000513, 7222300011, 7248600220, 3620000550]  
2                                       [2220000491]  
3  [2200000668, 2200000665, 2200000665, 300653144...  
4  [3890000619, 7231000105, 5100014297, 210000264...  
        HouseholdCode PurchaseWeek  \
308996       31226812   2015-11-23   
308997       31226812   2015-11-30   
308998       3122681

/Users/karthiksrinivasan/DD/Python/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
longest_weekly = np.max(house_prd_weekly.List_UPCs.apply(len))
print(longest_weekly)

46


In [12]:
trans_products_vocab = house_prd_weekly['List_UPCs']
word2vec_mb_weekly = gensim.models.Word2Vec(trans_products_vocab, sg = 1, size=300, window=longest_weekly, min_count=10, workers=4)

In [13]:
word2vec_mb_weekly.wv.save_word2vec_format('../data/MB_weekly_corpus_1000k_obs.txt')
# word2vec_mb_weekly.wv.save_word2vec_format('../data/MB_weekly_corpus_2015.txt')

In [ ]:
### Models for next week market basket prediction: 
## - Autoencoder: Feed the model one week's UPCs as embedding features (p = 300), and decode the next week's purchases
## - Autoencoder + Take into consideration Household info 
## - Recurrent neural network with bag of words approach (Still formulating method)
## - Graph embedding followed by graph-based prediction (Still formulating method)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(house_prd_weekly[["HouseholdCode", "List_UPCs"]], house_prd_weekly['FutureUPCs'], test_size=0.25, random_state=1234)
print(X_train.shape)
print(X_test.shape)

(231750, 2)
(77251, 2)


In [ ]:
### Simple auto-encoder